In [53]:
from pkg_req import *
from construction import *
from solver import *
from backend import *
from pulses import *
from fidelity import *
from qiskit.circuit import Parameter
from qiskit.pulse import acquire, MeasureChannel
from qiskit_algorithms.optimizers import NFT, ADAM
from qiskit.quantum_info import DensityMatrix

#test params
dim = 3
v0 = 4.86e9
anharm0 = -0.32e9
r0 = 0.22e9
v1 = 4.97e9
anharm1 = -0.32e9
r1 = 0.26e9
J = 0.002e9
dt= 1/4.5e9

shots=10000000 # Note: Fow now using a huge number so I can ensure counts in all 4 states. Need to add conditionals later
#Operators defined
H=hamiltonian(dim, v0, v1, anharm0, anharm1, J)
drive_op0=driveop(dim, r0)
drive_op1=driveop(dim, r1)

#solver
solver, solver_options=dynam_solver(H, drive_op0, drive_op1, v0, v1, dt)

#backend
backend=dynam_backend(solver, solver_options, dim)

des_state=np.zeros((4,4), dtype=complex) #|00> output
#des_state[1, 1] = 1  # Pure state |01>
des_state[3, 3] = 1  # Pure state |11>
des_state

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

In [64]:
def pulse_circ(num,A,sig,theta):
    schedules = []
    with pulse.build(backend=backend) as schedule:
        with pulse.align_sequential(): 
            g1=gauss(num,A,sig,theta)
            pulse.play(g1, pulse.DriveChannel(0)) 
            pulse.play(g1, pulse.DriveChannel(1))
            pulse.measure_all()
    schedules.append(schedule)
    job = backend.run(schedules, shots=shots)
    result = job.result()
    
    counts = {'00': int(result.get_counts(0)['00']), '01': int(result.get_counts(0)['01']), '10': int(result.get_counts(0)['10']), '11': int(result.get_counts(0)['11'])}  
    return counts

In [65]:
test=pulse_circ(20,1,10000,np.pi/3)
test

{'00': 7138234, '01': 3327, '10': 2844845, '11': 13594}

In [69]:
def calculate_fidelity_test(pulse_params):
    counts=pulse_circ(*pulse_params)
    print(counts)
    total_shots = sum(counts.values())
    probabilities = {state: count / total_shots for state, count in counts.items()}

    num_states = 4 
    measured_density_matrix = np.zeros((num_states, num_states), dtype=complex)
    states = ['00', '01', '10', '11']  
    for i, state in enumerate(states):
        prob = probabilities.get(state, 0)  # Prob for each state
        # |state><state|
        outer_product = np.outer(np.array([1 if j == i else 0 for j in range(num_states)]), 
                                 np.array([1 if j == i else 0 for j in range(num_states)]))
        measured_density_matrix += prob * outer_product

    #Convert to DensityMatrix object
    rho_measured = measured_density_matrix#DensityMatrix(measured_density_matrix)

    print('fidelity with target state set to |11>: ')
    fidelity = state_fidelity(rho_measured, des_state)
    return 1-np.abs(fidelity) 

In [70]:
te=[100,0.5,0.5,0]
calculate_fidelity_test(te)

{'00': 9729691, '01': 43, '10': 270260, '11': 6}
fidelity with target state set to |11>: 


np.float64(0.9999994)

In [73]:
optimizer = NFT(maxiter=10)#, maxfev=1024, disp=True,reset_interval=32)

#opt_params = optimizer.minimize(lambda x: calculate_fidelityx0(x, counts, ideal_density_matrix),x0=[100,0.5,5,0,0])
opt_params = optimizer.minimize(calculate_fidelity_test,x0=[100,0.7,0.7,np.pi])
#opt_vals = opt_params.x

UnassignedDurationError: 'All instruction durations should be assigned before creating `Schedule`.Please check `.parameters` to find unassigned parameter objects.'